In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from FRVRS import nu
import os.path as osp
import re
import os
import pandas as pd


# Analyze Util Function Usage Frequency

In [3]:

nu.delete_ipynb_checkpoint_folders()

In [8]:

# Create a regular expression of all the non-staticmethod-decorated utils functions
instance_defs_regex = re.compile(r'^    def ([a-z]+)([a-z_]+)\(\s*self,\s+([^\)]+)\):', re.MULTILINE)
functions_list = []
for file_path in ['frvrs_utils.py', 'notebook_utils.py']:
    with open(f'../py/FRVRS/{file_path}', 'r', encoding=nu.encoding_type) as f: text = f.read()
    for match_obj in instance_defs_regex.finditer(text):
        functions_list.append(''.join([s[0] for s in file_path.split('.')[0].split('_')]) + '\\.' + match_obj.group(1) + match_obj.group(2))

instance_functions_regex = re.compile('\\b(' + '|'.join(functions_list) + ')\\b')
all_calls_regex = re.compile('\\b[nf]u\\.[a-z_]+')

In [9]:

# Tabulate util function usage frequency
usage_frequency_dict = {}
all_calls_frequency_dict = {}
for sub_directory, directories_list, files_list in os.walk(nu.github_folder):
    if '.ipynb_checkpoints' not in sub_directory:
        for file_name in files_list:
            if file_name.endswith('.ipynb') and not ('Attic' in file_name):
                file_path = osp.join(sub_directory, file_name)
                with open(file_path, 'r', encoding=nu.encoding_type) as f:
                    lines_list = f.readlines()
                    for line in lines_list:
                        match_obj = instance_functions_regex.search(line)
                        if match_obj:
                            fn = match_obj.group(1)
                            usage_frequency_dict[fn] = usage_frequency_dict.get(fn, 0) + 1
                        match_obj = all_calls_regex.search(line)
                        if match_obj:
                            fn = match_obj.group()
                            all_calls_frequency_dict[fn] = all_calls_frequency_dict.get(fn, 0) + 1

In [10]:

usage_frequency_df = pd.Series(usage_frequency_dict).to_frame().reset_index().rename(columns={0: 'usage_frequency', 'index': 'function_call'})
usage_frequency_df.sort_values('usage_frequency', ascending=False).head(20)

,function_call,usage_frequency
11,fu.visualize_extreme_player_movement,36
4,fu.get_is_scene_aborted,25
5,nu.save_data_frames,20
13,nu.plot_sequence,17
25,fu.visualize_player_movement,15
12,fu.get_first_patient_interaction,14
26,fu.show_time_statistics,12
27,fu.plot_sequence_by_scene_tuple,10
17,nu.load_data_frames,9
21,nu.check_for_typos,8


In [11]:

all_calls_frequency_df = pd.Series(all_calls_frequency_dict).to_frame().reset_index().rename(columns={0: 'all_calls_frequency', 'index': 'function_call'})
all_calls_frequency_df.sort_values('all_calls_frequency', ascending=False).head(20)

,function_call,all_calls_frequency
14,fu.scene_groupby_columns,106
0,nu.pickle_exists,101
1,nu.load_object,101
3,nu.store_objects,85
2,fu.patient_groupby_columns,57
5,fu.get_session_groupby,39
44,fu.visualize_extreme_player_movement,36
20,nu.conjunctify_nouns,32
23,fu.get_logger_version,30
56,nu.get_sequences_by_count,27


In [ ]:

mask_series = (usage_frequency_df.usage_frequency == 1)
usage_frequency_df[mask_series]